# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [4]:
# Set the file path to import the WeatherPy_database.csv file
#file_path = r'C:/Users/snapi/Documents/KU Bootcamp Class Folder/all/World_Weather_Analysis/Vacation_Search'
# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(r'C:\Users\snapi\Documents\KU Bootcamp Class Folder\all\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jalu,LY,29.0331,21.5482,65.23,54,100,8.03,overcast clouds
1,1,Srandakan,ID,-7.9386,110.2506,74.62,92,100,5.95,light rain
2,2,Tasiilaq,GL,65.6145,-37.6368,23.68,62,100,6.11,overcast clouds
3,3,Santa Cruz,BO,-17.8000,-63.1667,104.05,20,4,14.97,clear sky
4,4,Geraldton,AU,-28.7667,114.6000,59.41,82,4,8.05,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [5]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 78
What is the maximum temperature you would like for your trip? 98


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [6]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Lorengau,PG,-2.0226,147.2712,81.16,72,28,6.04,scattered clouds
20,20,Sandakan,MY,5.8402,118.1179,78.48,94,20,2.30,few clouds
31,31,Calabozo,VE,8.9242,-67.4293,86.31,57,100,5.70,overcast clouds
32,32,Butaritari,KI,3.0707,172.7902,80.56,74,63,6.24,broken clouds
44,44,Avarua,CK,-21.2078,-159.7750,78.85,69,75,13.80,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [7]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
preferred_cities_df
clean_travel_cities = preferred_cities_df.dropna(how = 'any')
# Display sample data
preferred_cities_df.count()

City_ID                138
City                   138
Country                137
Lat                    138
Lng                    138
Max Temp               138
Humidity               138
Cloudiness             138
Wind Speed             138
Current Description    138
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [8]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
preferred_cities_df
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
9,Lorengau,PG,81.16,scattered clouds,-2.0226,147.2712
20,Sandakan,MY,78.48,few clouds,5.8402,118.1179
31,Calabozo,VE,86.31,overcast clouds,8.9242,-67.4293
32,Butaritari,KI,80.56,broken clouds,3.0707,172.7902
44,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [9]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Lorengau,PG,81.16,scattered clouds,-2.0226,147.2712,
20,Sandakan,MY,78.48,few clouds,5.8402,118.1179,
31,Calabozo,VE,86.31,overcast clouds,8.9242,-67.4293,
32,Butaritari,KI,80.56,broken clouds,3.0707,172.7902,
44,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
45,Mogadishu,SO,79.93,broken clouds,2.0371,45.3438,
46,Alofi,NU,80.49,scattered clouds,-19.0595,-169.9187,
67,Tiarei,PF,82.42,scattered clouds,-17.5333,-149.3333,
68,Arraial Do Cabo,BR,78.76,scattered clouds,-22.9661,-42.0278,
70,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [10]:
# Review the parameters to search for a hotel
from config import g_key
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [11]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels =requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [12]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Lorengau,PG,81.16,scattered clouds,-2.0226,147.2712,
20,Sandakan,MY,78.48,few clouds,5.8402,118.1179,
31,Calabozo,VE,86.31,overcast clouds,8.9242,-67.4293,
32,Butaritari,KI,80.56,broken clouds,3.0707,172.7902,
44,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [22]:
 import numpy as np
# Drop the rows where there is no Hotel Name.
hotel_df_nan = hotel_df[hotel_df['Hotel Name']==''] = np.nan

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,NaN,NaN,NaN,NaN,NaN,NaN,NaN
673,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [23]:
# In case of an empty DataFrame, load the sample data provided
clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
clean_hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [24]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [25]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [26]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Pudong Holiday Hotel</dd>\n<dt>City</dt><dd>Aksu</dd>\n<dt>Country</dt><dd>CN</dd>\n<dt>Current Weather</dt><dd>clear sky and 82.42 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>INN NEW HORIZON</dd>\n<dt>City</dt><dd>Touros</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>CCI Express Inn</dd>\n<dt>City</dt><dd>Morehead</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 75.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>39 On Nile Guest House</dd>\n<dt>City</dt><dd>Port Elizabeth</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 84.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Durrant House Hotel</dd>\n<dt>City</dt><dd>Northam</dd>\n<dt>Country</dt><dd>GB</dd>\n<dt>Current Weather</dt><dd>few clouds and 82.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Taj Krishna, Hyderabad</dd>\n<dt>Cit

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [27]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,41.12,80.26
1,-5.20,-35.46
2,37.27,-87.18
3,-33.92,25.57
4,51.03,-4.22
5,17.38,78.47
6,-33.59,26.89
7,-9.80,-139.03
8,22.08,-159.32
9,59.70,30.79


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [28]:
# Add a marker layer for each city to the map. 
# Create a figure to add the Google map as a layer
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))